In [46]:
import pandas as pd
import json

In [ ]:
# the xlsx cab be obtained at 
# https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/87YG4K
  # and then change the name to visits.xlsx and place it in the same folder as the notebook file
  # drop SourceLink columns
  # coerce errors in TripEndDate and TripDuration columns and other columns if data is not in
  # output as parquet to have better performance
# df = pd.read_excel("./visits.xlsx", index_col="TripID")
# df = df.drop([f"SourceLink{i}" for i in range(1, 34)], axis=1)
# df = df.drop([ "PriorityLeader", "CotravelHOGS"], axis=1)


# df['TripEndDate'] = pd.to_datetime(df['TripEndDate'], errors='coerce')
# df['TripStartDate'] = pd.to_datetime(df['TripStartDate'], errors='coerce')
# df['TripDuration'] = pd.to_numeric(df['TripDuration'], errors='coerce')

# df = df.replace({"Yes": True, "No": False})

# df.to_parquet('visits.parquet')

df = pd.read_parquet('visits.parquet')

/var/folders/hn/q5c1yk057dl151_qggqvl6dw0000gn/T/ipykernel_26042/234987465.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"Yes": True, "No": False})


In [52]:
df.columns

Index(['LeaderID', 'LeaderCountryOrIGO', 'LeaderCountryISO', 'LeaderRegion',
       'LeaderSubregion', 'LeaderRole', 'LeaderTitle', 'LeaderSurname',
       'LeaderFullName', 'Exiled', 'CountryVisited', 'CountryVisitedISO',
       'RegionVisited', 'SubRegionVisited', 'DisputedTerritory',
       'SemiAutoTerritory', 'TripYear', 'TripStartDate', 'TripEndDate',
       'TripDuration', 'SubHOGSTravelWithHOGS', 'MetHostHoGS', 'HostHOGSName',
       'SubHOGSmetHostSubHOGS', 'HostSubHOGSName', 'AttendedMultilatEvent',
       'NameMultilatEvent', 'AttendedMultilatMinisterialEvent',
       'NameMultilatMinisterialEvent', 'MetNonhostHOGS', 'NonhostHOGSNames',
       'PublicAddress', 'SignedAgreement', 'CulturalSiteOrCeremony',
       'BusinessLeaderOrForum', 'MetIGOLeader', 'IGOLeaderName', 'Notes',
       'Confidence(1-5)'],
      dtype='object')

In [49]:
visit_per_year = df["TripYear"].value_counts()
year_min_visits = visit_per_year.idxmin()
year_max_visits = visit_per_year.idxmax()
print(f"Year with least visits {year_min_visits}")
print(f"Year with max visits {year_max_visits}")


Year with least visits 2020
Year with max visits 2023


In [50]:
top_10_visits_by_country_2020 = df[df["TripYear"] == 2020]["LeaderCountryOrIGO"].value_counts().head(10)
top_10_visits_to_country_2020 = df[df["TripYear"] == 2020]["CountryVisited"].value_counts().head(10)

print(f"Top countries that sent Diplomatic visitors in 2020 {top_10_visits_by_country_2020}")
print(f"Top countries that received Diplomatic visitors in 2020 {top_10_visits_to_country_2020}")


KeyError: 'CountryVisited'

In [ ]:
most_visits_by_country_per_year = (df
  .groupby(["TripYear", "LeaderCountryOrIGO"])
  .size()
  .reset_index(name='count')
  .sort_values(["TripYear", "count"], ascending=[True,False])
  .groupby('TripYear')
  .head(1)
)
most_visits_to_country_per_year = (df
  .groupby(["TripYear", "CountryVisited"])
  .size()
  .reset_index(name='count')
  .sort_values(["TripYear", "count"], ascending=[True,False])
  .groupby('TripYear')
  .head(1)
)

sum_visits_by = (most_visits_by_country_per_year
  .groupby("LeaderCountryOrIGO")["TripYear"]
  .apply(lambda d: sorted(d))
  .reset_index()
)

sum_visits_to =(most_visits_to_country_per_year
  .groupby("CountryVisited")["TripYear"]
  .apply(lambda d: sorted(d))
  .reset_index()
)


,LeaderCountryOrIGO,TripYear
0,Bosnia and Herzegovina,[1993]
1,European Commission (EC),"[2013, 2021, 2022, 2023, 2024]"
2,France,[2018]
3,Greece,[1992]
4,Hungary,[2020]
5,Lithuania,[1995]
6,Nigeria,[2005]
7,North Atlantic Treaty Organization (NATO),"[1996, 1997, 2003, 2004]"
8,Organisation of African Unity (OAU),[1994]
9,Palestine,"[1990, 1991, 1999, 2000, 2001, 2006, 2007, 200..."


In [ ]:
sum_visits_by["NumYearsTop"] = sum_visits_by["TripYear"].apply(len)
sum_visits_by.sort_values("NumYearsTop", ascending=False)

,LeaderCountryOrIGO,TripYear,NumYearsTop
9,Palestine,"[1990, 1991, 1999, 2000, 2001, 2006, 2007, 200...",11
11,United Nations (UN),"[1998, 2002, 2011, 2014, 2015, 2016, 2017]",7
1,European Commission (EC),"[2013, 2021, 2022, 2023, 2024]",5
7,North Atlantic Treaty Organization (NATO),"[1996, 1997, 2003, 2004]",4
0,Bosnia and Herzegovina,[1993],1
2,France,[2018],1
3,Greece,[1992],1
4,Hungary,[2020],1
5,Lithuania,[1995],1
6,Nigeria,[2005],1


In [ ]:
sum_visits_to["NumYearsTop"] = sum_visits_to["TripYear"].apply(len)
sum_visits_to.sort_values("NumYearsTop", ascending=False)

,CountryVisited,TripYear,NumYearsTop
2,United States,"[1990, 1991, 1992, 1993, 1994, 1995, 1997, 199...",25
0,Belgium,"[2009, 2010, 2012, 2013, 2014, 2015, 2018, 202...",9
1,France,[1996],1


looking beyond 2020 and investigating other years it seems that Palestine has sent most diplomatic visitors in 11 out of 35 years in each year.

in terms of who received the most diplomatic visitors it seems that the United States has received the most diplomatic visitors in 25 out of 35 years.

In [ ]:
# check how were the visits over different regions
origin_of_visit = (df
  .groupby(["LeaderRegion", "TripYear"])
  .size()
)

destination_of_visit = (df
  .groupby(["RegionVisited", "TripYear"])
  .size()
)

origin_of_visit

LeaderRegion  TripYear
Africa        1990        325
              1991        295
              1992        235
              1993        215
              1994        281
                         ... 
Oceania       2020          8
              2021         22
              2022         75
              2023        130
              2024        128
Length: 245, dtype: int64

In [ ]:
# get the longest trip duration
longest_trip = df[df["TripDuration"] == df["TripDuration"].max()].T

# get the exchange relationship between the countries per year.
exchange_relationship = (df.
  groupby(["TripYear", "LeaderCountryOrIGO", "LeaderRegion", "CountryVisited",  "RegionVisited"])
  .size()
  .reset_index(name="Count")
)


exchange_relationship.head()


,TripYear,LeaderCountryOrIGO,LeaderRegion,CountryVisited,RegionVisited,Count
0,1990,Afghanistan,Asia,India,Asia,1
1,1990,Afghanistan,Asia,Soviet Union,Europe,1
2,1990,Afghanistan,Asia,Switzerland,Europe,1
3,1990,Albania,Europe,United States,Northern America,1
4,1990,Algeria,Africa,Egypt,Africa,1


In [ ]:
stacked_bar = (df
  .groupby(["TripYear", "LeaderRegion", "RegionVisited"])
  .size()
  .reset_index(name="Count")
)

visits_by_country_per_year = (df
  .groupby(["TripYear", "LeaderCountryOrIGO"])
  .size()
  .reset_index(name="count")
  .sort_values(['TripYear', 'count'], ascending=[True, False])
  # .to_parquet("top_sent.parquet")
)
visits_to_country_per_year = (df
  .groupby(["TripYear", "CountryVisited"])
  .size()
  .reset_index(name='count')
  .sort_values(["TripYear", "count"], ascending=[True,False])
  # .to_parquet("top_received.parquet")
)


visits_by_country_per_year

,TripYear,LeaderCountryOrIGO,count
119,1990,Palestine,115
116,1990,Organisation of African Unity (OAU),33
88,1990,Lithuania,31
43,1990,Egypt,25
147,1990,Sudan,24
...,...,...,...
6584,2024,Kiribati,1
6606,2024,Mexico,1
6612,2024,Morocco,1
6614,2024,Myanmar,1
